<div style="width: 100%; margin: 0 auto;">
    <a href="https://github.com/e10101/learning-operations-research">
        <img src="../assets/banner.svg" alt="Learning Operations Research" style="width: 100%; height: auto; display: block;">
    </a>
</div>

# Integer Programming - Machine Scheduling

---

[![Github](../assets/badges/github.svg)](https://github.com/e10101/learning-operations-research)


## Problem

### Problem Details

We need to schedule 4 jobs on a single machine. The processing time (in days) required for each job and its corresponding due date are listed below:

| Job | Processing Time (Days) | Due Date      |
| --- | ---------------------- | ------------- |
| 1   | 6                      | End of day 8  |
| 2   | 4                      | End of day 4  |
| 3   | 5                      | End of day 12 |
| 4   | 8                      | End of day 16 |

The objective is to find the sequence for processing these jobs that minimizes the total delay.



### Source

This problem is adapted from:

Winston, W.L. (2004). *Operations Research Applications and Algorithms* (4th ed.). Duxbury Press, Pacific Grove, CA. (Chapter 9: Integer Programming, p. 528).


## Create Model

In [1]:
import gurobipy as gp

In [2]:
m = gp.Model("Machine Scheduling")

Restricted license - for non-production use only - expires 2026-11-23


## Create Variables

In [4]:
JOBS = 4
QUEUES = 4

In [5]:
X = m.addVars(JOBS, QUEUES, vtype=gp.GRB.BINARY, name="X")

In [6]:
C = m.addVars(QUEUES, vtype=gp.GRB.INTEGER, name="C")

In [7]:
T = m.addVars(JOBS, vtype=gp.GRB.INTEGER, name="T")

## Create Constraints

In [9]:
for i in range(JOBS):
    m.addConstr(sum([X[i, j] for j in range(QUEUES)]) == 1, name=f"job_{i}")

In [10]:
for j in range(QUEUES):
    m.addConstr(sum([X[i, j] for i in range(JOBS)]) == 1, name=f"queue_{j}")

In [11]:
P = [6, 4, 5, 8]

In [12]:
for j in range(QUEUES):
    p_sum = sum([X[i, j] * P[i] for i in range(JOBS)])

    if j == 0:
        m.addConstr(p_sum == C[j], name=f"C_{j}")
    else:
        m.addConstr(C[j-1] + p_sum == C[j], name=f"C_{j}")

In [13]:
D = [8, 4, 12, 16]

In [14]:
M = sum(P)

In [15]:
for i in range(JOBS):
    for j in range(QUEUES):
        m.addConstr(T[i] >= C[j] - D[i] - M * (1 - X[i, j]), name=f"T_{i}_{j}")

## Set Objective

In [16]:
m.setObjective(sum([T[i] for i in range(JOBS)]), sense=gp.GRB.MINIMIZE)

## Solve

In [17]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (armlinux64 - "Debian GNU/Linux 11 (bullseye)")

CPU model: ARM64
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 28 rows, 24 columns and 103 nonzeros
Model fingerprint: 0x6dc8262d
Variable types: 0 continuous, 24 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 14.0000000
Presolve removed 11 rows and 6 columns
Presolve time: 0.00s
Presolved: 17 rows, 18 columns, 61 nonzeros
Variable types: 0 continuous, 18 integer (13 binary)

Root relaxation: objective 8.457143e+00, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    8.45714    0   11   14.00000    8.45714  39

## Result

In [18]:
m.getVars()

[<gurobi.Var X[0,0] (value 0.0)>,
 <gurobi.Var X[0,1] (value 1.0)>,
 <gurobi.Var X[0,2] (value 0.0)>,
 <gurobi.Var X[0,3] (value 0.0)>,
 <gurobi.Var X[1,0] (value 1.0)>,
 <gurobi.Var X[1,1] (value 0.0)>,
 <gurobi.Var X[1,2] (value -0.0)>,
 <gurobi.Var X[1,3] (value 0.0)>,
 <gurobi.Var X[2,0] (value 0.0)>,
 <gurobi.Var X[2,1] (value -0.0)>,
 <gurobi.Var X[2,2] (value 1.0)>,
 <gurobi.Var X[2,3] (value -0.0)>,
 <gurobi.Var X[3,0] (value -0.0)>,
 <gurobi.Var X[3,1] (value -0.0)>,
 <gurobi.Var X[3,2] (value 0.0)>,
 <gurobi.Var X[3,3] (value 1.0)>,
 <gurobi.Var C[0] (value 4.0)>,
 <gurobi.Var C[1] (value 10.0)>,
 <gurobi.Var C[2] (value 15.0)>,
 <gurobi.Var C[3] (value 23.0)>,
 <gurobi.Var T[0] (value 2.0)>,
 <gurobi.Var T[1] (value -0.0)>,
 <gurobi.Var T[2] (value 3.0)>,
 <gurobi.Var T[3] (value 7.0)>]

In [19]:
m.objVal

12.0